通过历史数据评估阳床树脂再生能力。

因所填表格不满足数据库第一范式，所以需要处理数据的大量工作。

从更换树脂当天至未来的某一天，当原树脂洗床所多消耗酸的成本可以覆盖更换树脂的成本时，就值得更换树脂。

清洗单个单元格

In [ ]:
import pandas as pd
import re
import calendar
import os
from time import sleep
from tqdm.auto import tqdm

# 初始化文件
if os.path.exists('RegenerationRawInfo.csv'):
    os.remove('RegenerationRawInfo.csv')

valid_columns = [2,5,8]
years = [2019,2020,2021,2022]

regeneration_df = pd.DataFrame({'date':[], 'regeneration':[]})
regeneration_df.to_csv("RegenerationRawInfo.csv",mode='a',index=False)


for year in tqdm(years,desc="year"):
    if (year < 2021):
        valid_index1 = 25
        valid_index2 = 26
    else:
        valid_index1 = 27
        valid_index2 = 28

    # 遍历路径文件
    for month in tqdm(range(1,13,1),desc="month"):
        file_name = '干熄焦生产记录台账/{0}/干熄焦生产记录台账（{1}月）.xlsx'.format(year,month)
        #file_date_info = re.findall(r'\d{1,}', file_name) # 提取路径文件信息(年、月）
        #month = file_date_info[1]
        # year = file_date_info[0]

        # 返回（当月总周数，当月总天数）
        month_range = calendar.monthrange(int(year),int(month))
        
        # 遍历表单
        for day in range(1,month_range[1]+1,1):  # 从第1天开始至当月最后一天
            try:
                regeneration_date = pd.read_excel(file_name, sheet_name = "{}日".format(day), header=None) # 读取当日表单
            except FileNotFoundError as file_error:
                print(file_error)
                break
            regeneration_cell = regeneration_date.iloc[valid_index1:valid_index2,[2,5,8]].astype(str) # 只保留包含洗床单元格的数据
            
            # 构建日期DataFrame
            regenerate_date = {'date':["{0}-{1}-{2}".format(year, month, day)]} # 再生日期信息
            regeneration_date_df = pd.DataFrame(data=regenerate_date)
        
            # 对单元格遍历，拆分、分割
            for valid_column in valid_columns:
                regeneration_split_cell = regeneration_cell[valid_column].T.str.split(r'\s+', expand=True) # 转置,（以至少1个空格)分割
            
                # 根据拆分的单元格长度，遍历追加入文件
                for l in range(int(regeneration_split_cell.size)):
                    regeneration_split_cell[l] =  regeneration_split_cell[l].astype(str)
                        
                    # 重建洗床信息DataFrame
                    regeneration_split_cell_value = regeneration_split_cell[l].values
                    regeneration_split_cell_value_df = pd.DataFrame(regeneration_split_cell_value)
                    # print(regeneration_split_cell_value_df)
                        
                    frames = [regeneration_date_df, regeneration_split_cell_value_df]
                    result = pd.concat(frames,join="outer",axis=1, ignore_index=True)
                    result.to_csv("RegenerationRawInfo.csv",mode='a',header=False, index=False)

# 剔除空行
regeneration_raw_info = pd.read_csv("RegenerationRawInfo.csv")
regeneration_raw_info.dropna(axis='rows', inplace=True)
regeneration_raw_info.reset_index(drop=True, inplace=True)  # drop=True能避免把旧列插入重排列

regeneration_raw_info.to_csv("RegenerationRawInfo.csv",mode='w', index=False)

year:   0%|          | 0/4 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

细分洗床数据

In [ ]:
rg_info = pd.read_csv("RegenerationRawInfo.csv")
pd.to_datetime(rg_info.date)

rg_detail = rg_info['regeneration'].str.split(r'分? ?再生|，|进|置换', regex=True, expand=True)
print(rg_detail)
rs = pd.concat([rg_info.date,rg_detail[0], rg_detail[1]], axis=1)
rs.to_csv("test.csv",index=False)

1#阳床

In [ ]:
fcation_resin_1 = rs.loc[rs.iloc[:,2].str.contains(r'1#阳',regex=True, na=False)]
fcation_resin_1.to_csv("fcation_resin_1.csv",index=False)

2#阳床

In [ ]:
fcation_resin_2 = rs.loc[rs.iloc[:,2].str.contains(r'2#阳',regex=True, na=False)]
fcation_resin_2.to_csv("fcation_resin_2.csv",index=False)

3#阳床

In [ ]:
fcation_resin_3 = rs.loc[rs.iloc[:,2].str.contains(r'3#阳',regex=True, na=False)]
fcation_resin_3.to_csv("fcation_resin_3.csv",index=False)